In [ ]:
prec = 30

In [ ]:
cvars = []

def cvar(u,x):
    return cvars[u-1][x]

def c1(x):
    return cvar(1,x)
def c2(x):
    return cvar(2,x)
def c3(x):
    return cvar(3,x)
def c4(x):
    return cvar(4,x)
def c5(x):
    return cvar(5,x)
def c6(x):
    return cvar(6,x)

In [ ]:
def compute_complexity(N, x, generate_variables, get_boundary_conditions, generate_equations, ticks=False, start=None):
    global cvars
    cvars = generate_variables(N)
    # Knowledge base -- dynamic programming memory
    KB = get_boundary_conditions(N)
    
    if ticks:
        print 'x     log(guesses)'
        print '-' * 30
    
    if start is None:
        start = N-1    
    
    for i in range(start, x-1, -1):
        equations, variables = generate_equations(N, i, KB)
        solutions = solve(equations, variables, solution_dict=True)[0]
        # add computed values
        for v in variables:
            KB[v] = solutions[v]
        if ticks:
            print '{:3d} {:10.2f}'.format(i, float(log(KB[c1(i)], 2).n(prec)))
    return KB[cvar(1, x)]

In [ ]:
# RC4   
def generate_rc4_variables(N):
    variables = [[var('c_' + str(u) + '_' + str(x)) for x in range(N+2)] for u in range(3)]
    return variables
    
def get_rc4_boundary_conditions(N):
    KB = {}
    for i in range(3):
        KB[cvar(i,N)] = 0
        KB[cvar(i,N+1)] = 0
    return KB
    
def generate_rc4_simple_equations(N, x, KB):
    eq1 = c1(x) == x/N * c2(x) + (1 - x/N) * (N-x) * (1 + KB[c2(x+1)])
    eq2 = c2(x) == x/N * c3(x) + (1 - x/N) * (N-x) * (1 + KB[c3(x+1)])
    eq3 = c3(x) == x/N * (1/N * c1(x)) + (1 - x/N)^2 * (1 + KB[c1(x+1)])
    return ([eq1, eq2, eq3], [cvar(i,x) for i in range(1, 4)])
             
def generate_rc4_change_order_equations(N, x, KB):
    eq1 = c1(x) == x/N * c2(x) + (1 - x/N) * (N-x) * (1 + KB[c2(x+1)])
    eq2 = c2(x) == x/N * ((1 - x/N)^2 * (1 + KB[c1(x+1)]) + x/N * 1/N * c1(x)) + (1 - x/N) * c3(x)
    
    Sj_unknown = 1 + 1/N * KB[c1(x+1)] + (N-x-1) * (1 - (x+1)/N) * (1 + KB[c1(x+2)])
    Sj_known = (1 + KB[c1(x+1)])
    eq3 = c3(x) == (1 - x/N) *  Sj_unknown + x/N * (1 - x/N) * Sj_known
    
    return ([eq1, eq2, eq3], [cvar(i,x) for i in range(1, 4)])
    
def e(N, x):
    return (1 - (x+1)/N) * (1 - 1/(N-x))

def f(N, x):
    return (N-x) * (1 + e(N,x)) + x/N 
    
def generate_rc4_knudsen_equations(N, x, KB):
    eq1 = c1(x) == x/N * c2(x) + (1 - x/N) * (N-x) * KB[c2(x+1)]
    eq2 = c2(x) == x/N * ((1 - x/N)^2 * (1 + KB[c1(x+1)]) + 1/N * c1(x)) + (1 - x/N) * c3(x)
    eq3 = c3(x) == (1 - x/N) * (f(N,x) + (2*x+1)/N * KB[c1(x+1)] + (N-x) * e(N,x) * KB[c1(x+2)])
    return ([eq1, eq2, eq3], [cvar(i,x) for i in range(1, 4)])

In [ ]:
# test: RC4
N, x = 4, 3
result = compute_complexity(
    N, x,
    generate_rc4_variables,
    get_rc4_boundary_conditions,
    generate_rc4_simple_equations
)
print '{:g}'.format( float(result) )
print '{:.2f}'.format( float(log(result, 2).n(prec)) )

In [ ]:
params = [8, 12, 16, 64, 128, 256]
print 'simple'
for N in params:
    result = compute_complexity(
        N, 0,
        generate_rc4_variables,
        get_rc4_boundary_conditions,
        generate_rc4_simple_equations
    )
    print '{:3},{:3}: {:>7.2f}'.format(N, 0, float(log(result, 2).n(prec)) )

print 'change order'
for N in params:
    result = compute_complexity(
        N, 0,
        generate_rc4_variables,
        get_rc4_boundary_conditions,
        generate_rc4_change_order_equations
    )
    print '{:3},{:3}: {:>7.2f}'.format(N, 0, float(log(result, 2).n(prec)) )

print 'knudsen'
for N in params:
    result = compute_complexity(
        N, 0,
        generate_rc4_variables,
        get_rc4_boundary_conditions,
        generate_rc4_knudsen_equations
    )
    print '{:3},{:3}: {:>7.2f}'.format(N, 0, float(log(result, 2).n(prec)) )

In [ ]:
# Spritz
def generate_spritz_variables(N):
    variables = [[var('c_' + str(u) + '_' + str(x)) for x in range(N+2)] for u in range(6)]
    return variables
  
def get_spritz_boundary_conditions(N):
    KB = {}
    for i in range(6):
        KB[cvar(i,N)] = 0
        KB[cvar(i,N+1)] = 0
    return KB
                
def generate_spritz_simple_equations(N, x, KB):
    eqs = []
    for i in range(1, 6):
        eqs.append(
            cvar(i,x) == x/N * cvar(i+1,x) + (1 - x/N) * (N-x) * (1 + KB[cvar(i+1,x+1)])
        )
    eq6 = c6(x) == x/N * (1/N * c1(x)) + (1 - x/N)^2 * (1 + KB[c1(x+1)])
    eqs.append(eq6)
    return (eqs, [cvar(i,x) for i in range(1, 7)])
             
def generate_spritz_change_order_equations(N, x, KB):
    eqs = []
    for i in range(1, 5):
        eqs.append(
            cvar(i,x) == x/N * cvar(i+1,x) + (1 - x/N) * (N-x) * (1 + KB[cvar(i+1,x+1)])
        )
    eq5 = c5(x) == x/N * ((1 - x/N)^2 * (1 + KB[c1(x+1)]) + x/N * 1/N * c1(x)) + (1 - x/N) * c6(x)
    eq6 = c6(x) == (1 - x/N) * (1 + 1/N * KB[c1(x+1)] + (N-x-1) * (1 - (x+1)/N) * (1 + KB[c1(x+2)])) + x/N * (1 - x/N) * (1 + KB[c1(x+1)])
    eqs += [eq5, eq6]
    return (eqs, [cvar(i,x) for i in range(1, 7)])

def generate_spritz_ankele_equations(N, x, KB):
    eqs = []
    for i in range(1, 6):
        eqs.append(
            cvar(i,x) == x/N * cvar(i+1,x) + (1 - x/N) * (N-x) * KB[cvar(i+1,x+1)]
        )
    eq6 = c6(x) == x/N * ((1 - x/N) + c1(x)) + (1 - x/N) * (x/N + KB[c1(x+1)])
    eqs.append(eq6)
    return (eqs, [cvar(i,x) for i in range(1, 7)])

In [ ]:
# test: Spritz
N, x = 16, 0
result = compute_complexity(
    N, x,
    generate_spritz_variables,
    get_spritz_boundary_conditions,
    generate_spritz_change_order_equations
)
print '{:g}'.format( float(result) )
print '{:.2f}'.format( float(log(result, 2).n(prec)) )

In [ ]:
params = [(8,0), (10,0), (16,5), (64,48), (128,114), (256,240), (64,0), (128,0), (256,0)]
print 'simple'
for N, x in params:
    result = compute_complexity(
        N, x,
        generate_spritz_variables,
        get_spritz_boundary_conditions,
        generate_spritz_simple_equations
    )
    print '{:3},{:3}: {:>7.2f}'.format(N, x, float(log(result, 2).n(prec)) )

print 'ankele'
for N, x in params:
    result = compute_complexity(
        N, x,
        generate_spritz_variables,
        get_spritz_boundary_conditions,
        generate_spritz_ankele_equations
    )
    print '{:3},{:3}: {:>7.2f}'.format(N, x, float(log(result, 2).n(prec)) )

print 'change order'
for N, x in params:
    result = compute_complexity(
        N, x,
        generate_spritz_variables,
        get_spritz_boundary_conditions,
        generate_spritz_change_order_equations
    )
    print '{:3},{:3}: {:>7.2f}'.format(N, x, float(log(result, 2).n(prec)) )

In [ ]:
# Special -- Banik et al
def generate_spritz_banik_variables(N):
    variables = [[var('c_' + str(u) + '_' + str(x)) for x in range(N/2+2)] for u in range(14)]
    return variables
    
def get_spritz_banik_boundary_conditions(N):
    KB = {}
    for i in range(14):
        KB[cvar(i,N/2-1)] = 1
        KB[cvar(i,N/2)] = 0
    return KB    
    
def generate_spritz_special_T1_equations(N, x, KB):
    N2 = N/2
    eqs = []
    for i in set(range(1,11))-set([5,10]):
        eqs.append(
            cvar(i,x) == x/N2 * cvar(i+1,x) + (1 - x/N2) * (N2-x) * KB[cvar(i+1,x+1)]
        )
    eq5 = cvar(5,x) == (x/N2)^2 * cvar(6,x) + (1 - x/N2)^2 * KB[cvar(6,x+1)]
    eq10 = cvar(10,x) == (x/N2)^2 * cvar(1,x) + (1 - x/N2)^2 * KB[cvar(1,x+1)]
    eqs += [eq5,eq10]
    return (eqs, [cvar(i,x) for i in range(1, 11)])
    
def generate_spritz_special_T2_equations(N, x, KB):
    N2 = N/2
    eqs = []
    for i in set(range(1,14))-set([6,10]):
        eqs.append(
            cvar(i,x) == x/N2 * cvar(i+1,x) + (1 - x/N2) * (N2-x) * KB[cvar(i+1,x+1)]
        )
    eq14 = cvar(14,x) == x/N2 * cvar(1,x) + (1 - x/N2) * (N2-x) * KB[cvar(1,x+1)]
    eq6 = cvar(6,x) == (x/N2)^2 * cvar(7,x) + (1 - x/N2)^2 * KB[cvar(7,x+1)]
    eq10 = cvar(10,x) == (x/N2)^2 * cvar(11,x) + (1 - x/N2)^2 * KB[cvar(11,x+1)]
    eqs += [eq6,eq10,eq14]
    return (eqs, [cvar(i,x) for i in range(1, 15)])

In [ ]:
# test: Special -- Banik -- T1*T2
N, x = 16, 0
t1 = compute_complexity(
    N, x,
    generate_spritz_banik_variables,
    get_spritz_banik_boundary_conditions,
    generate_spritz_special_T1_equations,
    start=N/2-2
)
t2 = compute_complexity(
    N, x,
    generate_spritz_banik_variables,
    get_spritz_banik_boundary_conditions,
    generate_spritz_special_T2_equations,
    start=N/2-2
)
print '{:g}'.format( float(t1*t2) )
print '{:.2f}'.format( float(log(t1*t2, 2).n(prec)) )

In [ ]:
params = [(16,0), (18,0), (20,0), (64,22), (128,50), (256,110), (64,0), (128,0), (256,0)]
for N, x in params:
    t1 = compute_complexity(
        N, x,
        generate_spritz_banik_variables,
        get_spritz_banik_boundary_conditions,
        generate_spritz_special_T1_equations,
        start=N/2-2
    )
    t2 = compute_complexity(
        N, x,
        generate_spritz_banik_variables,
        get_spritz_banik_boundary_conditions,
        generate_spritz_special_T2_equations,
        start=N/2-2
    )
    print '{:3},{:3}: {:>7.2f}'.format(N, x, float(log(t1*t2, 2).n(prec)) )

In [ ]:
# Special -- our analysis
def cvar(u,v,x,y):
    return cvars[u-1][v][x][y]

def c1(v,x,y):
    return cvar(1,v,x,y)
def c2(v,x,y):
    return cvar(2,v,x,y)
def c3(v,x,y):
    return cvar(3,v,x,y)
def c4(v,x,y):
    return cvar(4,v,x,y)
def c5(v,x,y):
    return cvar(5,v,x,y)
def c6(v,x,y):
    return cvar(6,v,x,y)
    
def generate_spritz_special_variables(N):
    cvars = [[[[var('c_'+str(u)+'_'+str(v)+'_'+str(x)+'_'+str(y)) 
                for y in range(N/2+2)] for x in range(N/2+2)] for v in range(4)] for u in range(6)]
    return cvars
    
def get_spritz_special_boundary_conditions(N):
    N2 = N/2
    KB = {}
    # Boundary conditions
    for u in range(1,7):
        for v in range(4):
            KB[cvar(u,v,N2,N2)] = 0
            for i in range(N2+1):
                KB[cvar(u,v,i,N2+1)] = 0
                KB[cvar(u,v,N2+1,i)] = 0
    return KB
    
# generate equations for given a,b and with data from the KB
def generate_spritz_special_equations(N, x, y, KB):
    N2 = N/2
    eq = []
    va = [c1(j,x,y) for j in range(4)]
    va += [c2(j,x,y) for j in range(4)]
    va += [c3(j,x,y) for j in range(4)]
    va += [c4(j,x,y) for j in range(4)]
    va += [c5(j,x,y) for j in range(4)]
    va += [c6(j,x,y) for j in range(4)]
    
    # STEP 1 (guessing S[i], parity of the index: 1,0,1,0)
    eq.append(c1(0,x,y) == y/N2*c2(0,x,y) + (1-y/N2)*(N2-y)*(1+KB.get(c2(0,x,y+1))))
    eq.append(c1(1,x,y) == x/N2*c2(1,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c2(1,x+1,y))))
    eq.append(c1(2,x,y) == y/N2*c2(2,x,y) + (1-y/N2)*(N2-y)*(1+KB.get(c2(2,x,y+1))))
    eq.append(c1(3,x,y) == x/N2*c2(3,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c2(3,x+1,y))))
    
    # STEP 2 (guessing S[j+S[i]], parity of the index: 0,0,0,0)
    for v in range(4):
        eq.append(c2(v,x,y) == x/N2*c3(v,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c3(v,x+1,y))))

    # STEP 3 (guessing S[j], parity of the index: 1,0,1,0)
    eq.append(c3(0,x,y) == y/N2*c4(0,x,y) + (1-y/N2)*(N2-y)*(1+KB.get(c4(0,x,y+1))))
    eq.append(c3(1,x,y) == x/N2*c4(1,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c4(1,x+1,y))))
    eq.append(c3(2,x,y) == y/N2*c4(2,x,y) + (1-y/N2)*(N2-y)*(1+KB.get(c4(2,x,y+1))))
    eq.append(c3(3,x,y) == x/N2*c4(3,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c4(3,x+1,y))))

    # STEP 4 (guessing S[z+k], parity of the index: 1,0,0,1)
    eq.append(c4(0,x,y) == y/N2*c5(0,x,y) + (1-y/N2)*(N2-y)*(1+KB.get(c5(0,x,y+1))))
    eq.append(c4(1,x,y) == x/N2*c5(1,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c5(1,x+1,y))))
    eq.append(c4(2,x,y) == x/N2*c5(2,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c5(2,x+1,y))))
    eq.append(c4(3,x,y) == y/N2*c5(3,x,y) + (1-y/N2)*(N2-y)*(1+KB.get(c5(3,x,y+1))))

    # STEP 5 (guessing S[i+S[z+k]], parity of the index: 1,1,0,0)
    eq.append(c5(0,x,y) == y/N2*c6(0,x,y) + (1-y/N2)*(N2-y)*(1+KB.get(c6(0,x,y+1))))
    eq.append(c5(1,x,y) == y/N2*c6(1,x,y) + (1-y/N2)*(N2-y)*(1+KB.get(c6(1,x,y+1))))
    eq.append(c5(2,x,y) == x/N2*c6(2,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c6(2,x+1,y))))
    eq.append(c5(3,x,y) == x/N2*c6(3,x,y) + (1-x/N2)*(N2-x)*(1+KB.get(c6(3,x+1,y))))

    # STEP 6(guessing S[j+S[i+S[z+k]]], parity of the index: 1,0,0,1)
    eq.append(c6(0,x,y) == y/N2*1/N2*c1(1,x,y) + (1-y/N2)^2*(1+KB.get(c1(1,x,y+1))))
    eq.append(c6(1,x,y) == x/N2*1/N2*c1(2,x,y) + (1-x/N2)^2*(1+KB.get(c1(2,x+1,y))))
    eq.append(c6(2,x,y) == x/N2*1/N2*c1(3,x,y) + (1-x/N2)^2*(1+KB.get(c1(3,x+1,y))))
    eq.append(c6(3,x,y) == y/N2*1/N2*c1(0,x,y) + (1-y/N2)^2*(1+KB.get(c1(0,x,y+1))))

    return (eq, va)

In [ ]:
def compute_spritz_special_complexity(N, even=0, odd=0, ticks=False):
    global cvars
    cvars = generate_spritz_special_variables(N)
    
    KB = get_spritz_special_boundary_conditions(N)

    N2 = N/2
    for x in range(N2,-1,-1):
        if ticks:
            print 'x=', x, '-> y: ',
        for y in range(N2,x-1,-1):
            if ticks:
                print y,
            # get equations for x, y
            eqs, eqvars = generate_spritz_special_equations(N, x, y, KB)
            sol = solve(eqs, eqvars, solution_dict=True)[0]
            # expand KB
            for v in eqvars:
                KB[v] = sol[v].n(prec)
            if y == x:
                if ticks:
                    print
                break
            # get equations for y,x
            eqs, eqvars = generate_spritz_special_equations(N, y, x, KB)
            sol = solve(eqs, eqvars, solution_dict=True)[0]
            # expand KB
            for v in eqvars:
                KB[v] = sol[v].n(prec)
    
    return KB[c1(0,even,odd)]

In [ ]:
# test: Special -- our analysis
N, even, odd = 10, 0, 0
result = compute_spritz_special_complexity(N, even, odd)
print '{:g}'.format( float(result) )
print '{:.2f}'.format( float(log(result, 2).n(prec)) )

In [ ]:
params = [(16,0,0), (18,0,0), (20,0,0), (64,22,22), (128,50,50), (256,110,110), (64,0,0), (128,0,0), (256,0,0)]
for N, x, y in params:
    result = compute_spritz_special_complexity(N, x, y)
    print '{},{:3},{:3}: {:>7.2f}'.format(N, x, y, float(log(result, 2).n(prec)) )